In [4]:
!pip install sqlcipher3-wheels



   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 16.6 MB/s eta 0:00:00


In [12]:
import os
import requests
from sqlcipher3 import dbapi2 as sqlite

# Input parameters
file_ids = [320,104]
base_url = "https://nsurvey.wbsedcl.in/tuntuni/dwnlds/nsurvey/"
password = "gavdwe erew rew24@#% 353%#$5 64&!2"

# Use Downloads folder
downloads_dir = os.path.join(os.path.expanduser("~"), "Downloads")
os.makedirs(downloads_dir, exist_ok=True)

# Delete SQLs
delete_queries = [
    "DELETE FROM txn_feature_point WHERE ly_key='p0';",
    "DELETE FROM txn_feature_line WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);",
    "DELETE FROM txn_lines WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);",
    "DELETE FROM txn_line2fdr WHERE typ=5;"
]

for fid in file_ids:
    fname = f"map-{fid}.dbn"
    url = base_url + fname
    file_path = os.path.join(downloads_dir, fname)

    resp = requests.get(url)
    if resp.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(resp.content)
        print("Downloaded:", file_path)

        # Open encrypted DB and run deletes
        conn = sqlite.connect(file_path)
        cur = conn.cursor()
        cur.execute(f"PRAGMA key = '{password}';")
        cur.execute("PRAGMA cipher_compatibility = 3;")

        try:
            for query in delete_queries:
                cur.execute(query)
                print("Executed:", query.strip())
            conn.commit()
            print(f"Deletions committed in {file_path}")
        except Exception as e:
            print(f"Error during deletion in {file_path}:", e)
        finally:
            conn.close()
    else:
        print("Failed to download:", fname, "Status code:", resp.status_code)


Downloaded: C:\Users\KRISHNENDU\Downloads\map-320.dbn
Executed: DELETE FROM txn_feature_point WHERE ly_key='p0';
Executed: DELETE FROM txn_feature_line WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);
Executed: DELETE FROM txn_lines WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);
Executed: DELETE FROM txn_line2fdr WHERE typ=5;
Deletions committed in C:\Users\KRISHNENDU\Downloads\map-320.dbn
Downloaded: C:\Users\KRISHNENDU\Downloads\map-104.dbn
Executed: DELETE FROM txn_feature_point WHERE ly_key='p0';
Executed: DELETE FROM txn_feature_line WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);
Executed: DELETE FROM txn_lines WHERE ln_id IN (SELECT ln_id FROM txn_line2fdr WHERE typ=5);
Executed: DELETE FROM txn_line2fdr WHERE typ=5;
Deletions committed in C:\Users\KRISHNENDU\Downloads\map-104.dbn
